In [26]:
import os

import librosa, librosa.display
import IPython.display as ipd
from random import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import signal
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

PROCESSED_DATA_PICKLE_PATH = '../data/df_all.pkl'

EMOTIONS = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

MFCC_LENGTH = 125

In [2]:
df = pd.read_pickle(PROCESSED_DATA_PICKLE_PATH)

def load_data(data_path):
    df = pd.read_pickle(data_path)

    # clean up data
    vals = []
    for i, val in enumerate(df["mfcc"].to_numpy()):
        if len(val) < MFCC_LENGTH:
            vals.append(i)

    df = df.drop(vals)
    df = df.reset_index(drop=True)

    X = np.stack(df["mfcc"].to_numpy())
    y = np.stack(df["emotion"].to_numpy())

    return X, y

In [39]:
def prepare_datasets(test_size):

    # load data
    X, y = load_data(PROCESSED_DATA_PICKLE_PATH)

    new_X = []
    for i in X:
        avg_mfcc = []
        for j in i:
            avg_mfcc.append(j.mean())
        new_X.append(avg_mfcc)
    X = np.array(new_X)

    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = prepare_datasets(0.3)

In [40]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)

print(X_test.shape)

y_pred = clf.predict(X_test)

print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))


(432, 125)
Accuracy: 0.3680555555555556
